In [155]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(work_path)

from helper.regrassion import *
from helper.util import *
from helper.plot import *
from helper.analysis import *


### [2] 데이터 가져오기

중고차 가격 예측을 위한 데이터 세트

https://www.kaggle.com/datasets/rishabhkarn/used-car-dataset

| 변수명 | 설명 |
|:--:|--:|
| car_name | Name of the car |
| registration_year | Vehicle registration year |
| insurance_validity | Type of insurance |
| fuel_type | Type of fuel |
| seats | Number of seats in the car |
| kms_driven | Total kms driven |
| ownsership | Number of ownership |
| transmission | Transmission mode |
| manufacturing_year | manufacturing year |
| mileage(kmpl) | Mileage of the car |
| engine(cc) | Engine capacity |
| max_power(bhp) | max power |
| torque(Nm) | torque |
| price(in lakhs) | Price of the used car |

In [156]:
origin = my_read_data("./res/Used_Car_Dataset.csv",info= False).drop('Unnamed: 0',axis=1)
df=origin.drop_duplicates()
df['price'] = df['price(in lakhs)']

In [157]:
df2 = my_replace_outliner_to_nan(df.drop('price(in lakhs)',axis=1),'price').dropna()
df2.head()

,seats,kms_driven,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price,car_name,registration_year,insurance_validity,fuel_type,ownsership,transmission,manufacturing_year
1,5,30615,17.40,999.0,999.0,9863.0,8.99,2020 Nissan Magnite Turbo CVT XV Premium Opt BSVI,Jan-21,Comprehensive,Petrol,First Owner,Automatic,2020
2,5,24000,20.68,1995.0,1995.0,188.0,23.75,2018 BMW X1 sDrive 20d xLine,Sep-18,Comprehensive,Diesel,First Owner,Automatic,2018
3,5,18378,16.50,1353.0,1353.0,13808.0,13.56,2019 Kia Seltos GTX Plus,Dec-19,Comprehensive,Petrol,First Owner,Manual,2019
4,5,44900,14.67,1798.0,1798.0,17746.0,24.00,2019 Skoda Superb LK 1.8 TSI AT,Aug-19,Comprehensive,Petrol,First Owner,Automatic,2019
5,5,42000,18.70,1199.0,1199.0,887.0,5.45,2017 Honda Jazz 1.2 S i VTEC,Jul-17,Third Party insurance,Petrol,First Owner,Manual,2017


In [158]:
brand = []
for i in df2['car_name']:
    brand.append(i.split(' ')[1])
df2['brand'] = brand
df2['brand'].unique()

array(['Nissan', 'BMW', 'Kia', 'Skoda', 'Honda', 'Hyundai', 'Tata',
       'Renault', 'Ford', 'Jeep', 'Maruti', 'Audi', 'Toyota', 'Jaguar',
       'Volkswagen', 'Mahindra', 'Mercedes-Benz', 'MG', 'Isuzu', 'Volvo',
       'Mitsubishi', 'Land', 'Porsche', 'Datsun', 'Fiat', 'Lamborghini'],
      dtype=object)

In [159]:
A = []
A_ = []
for i in origin['registration_year'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
# A, A_

In [160]:
origin['registration_year'].unique()

map_registration_year = {a:None for a in A}
for i in A_:
    map_registration_year[i] = i
df2['registration_year'] = df2['registration_year'].map(map_registration_year)
# df['registration_year']

In [161]:
month_dict = {'Apr':4,
 'Aug':8,
 'Feb':2,
 'Dec':12,
 'Jan':1,
 'Jul':7,
 'Jun':6,
 'Mar':3,
 'May':5,
 'Nov':11,
 'Oct':10,
 'Sep':9}

In [162]:
month = []
year = []

for i in df2['registration_year']:
    if i:
        try:
            month.append(month_dict[i[:3]])
            year.append(int(i[-2:]))
        except :
            month.append(None)
            year.append(None)
    else:
        month.append(None)
        year.append(None)

In [163]:
df2['month']=month
df2['year']=year

In [164]:
origin['insurance_validity'].value_counts()
map_insurance_validity = {a:a for a in origin['insurance_validity'].unique()}
map_insurance_validity['Petrol'] = None
map_insurance_validity['Not Available'] = None
map_insurance_validity['Third Party'] = 'Third Party insurance'
df2['insurance_validity'] = df2['insurance_validity'].map(map_insurance_validity)
df2['insurance_validity']

1               Comprehensive
2               Comprehensive
3               Comprehensive
4               Comprehensive
5       Third Party insurance
                ...          
1548            Comprehensive
1549    Third Party insurance
1550            Comprehensive
1551            Comprehensive
1552            Comprehensive
Name: insurance_validity, Length: 984, dtype: object

In [165]:
origin['fuel_type'].value_counts()
map_fuel_type = {a:a for a in origin['fuel_type'].unique()}
map_fuel_type['CNG'] = None
map_fuel_type['5 Seats'] = None
df2['fuel_type'] = df2['fuel_type'].map(map_fuel_type)
df2['fuel_type']

1       Petrol
2       Diesel
3       Petrol
4       Petrol
5       Petrol
         ...  
1548    Diesel
1549    Petrol
1550    Petrol
1551    Petrol
1552    Petrol
Name: fuel_type, Length: 984, dtype: object

In [166]:
origin['seats'].value_counts()
map_seats = {a:a if a<10 else None for a in origin['seats'].unique()}
df2['seats'] = df2['seats'].map(map_seats)
df2['seats']

1       5.0
2       5.0
3       5.0
4       5.0
5       5.0
       ... 
1548    5.0
1549    5.0
1550    5.0
1551    5.0
1552    5.0
Name: seats, Length: 984, dtype: float64

In [167]:
origin['ownsership'].value_counts()
map_ownsership = {a:a if "Owner" in a else None for a in origin['ownsership'].unique()}
df2['ownsership'] = df2['ownsership'].map(map_ownsership)
df2['ownsership']


1        First Owner
2        First Owner
3        First Owner
4        First Owner
5        First Owner
            ...     
1548     First Owner
1549            None
1550     First Owner
1551    Second Owner
1552     First Owner
Name: ownsership, Length: 984, dtype: object

In [168]:
A = []
A_ = []
for i in origin['transmission'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
origin['transmission'].value_counts()

Manual                 835
Automatic              668
2017                    28
2014                     5
2011                     3
2023                     2
2020                     2
2021                     2
2022                     2
Power Windows Front      2
2018                     2
2015                     1
2016                     1
Name: transmission, dtype: int64

In [169]:
map_transmission = {a:None for a in A}
for i in A_:
    map_transmission[i] = i
map_transmission['Power Windows Front']=None
df2['transmission'] = df2['transmission'].map(map_transmission)
df2['transmission']

1       Automatic
2       Automatic
3          Manual
4       Automatic
5          Manual
          ...    
1548    Automatic
1549         None
1550       Manual
1551       Manual
1552       Manual
Name: transmission, Length: 984, dtype: object

In [170]:
origin['manufacturing_year'].value_counts()
A = []
A_ = []
for i in origin['manufacturing_year'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
A, A_

(['2017',
  '2020',
  '2018',
  '2019',
  '2015',
  '2021',
  '2014',
  '2022',
  '2010',
  '2011',
  '2023',
  '2016',
  '2009',
  '2013',
  '2012',
  '2007'],
 ['Power Steering', 'Power Windows Front', 'Air Conditioner'])

In [171]:
map_manufacturing_year = {a:a for a in A}
for i in A_:
    map_manufacturing_year[i] = None
df2['manufacturing_year'] = df2['manufacturing_year'].map(map_manufacturing_year)
df2['manufacturing_year']

1       2020
2       2018
3       2019
4       2019
5       2017
        ... 
1548    2020
1549    None
1550    2017
1551    2018
1552    2018
Name: manufacturing_year, Length: 984, dtype: object

In [172]:
for i,v in enumerate(origin['mileage(kmpl)']):
    if v>40 : df2['mileage(kmpl)'][i] = None


In [173]:
max = 13519
min = 352

for i,v in enumerate(origin['engine(cc)']):
    if v < min or v > max : df2['engine(cc)'][i] = None


In [174]:
df2.dtypes

seats                 float64
kms_driven              int64
mileage(kmpl)         float64
engine(cc)            float64
max_power(bhp)        float64
torque(Nm)            float64
price                 float64
car_name               object
registration_year      object
insurance_validity     object
fuel_type              object
ownsership             object
transmission           object
manufacturing_year     object
brand                  object
month                 float64
year                  float64
dtype: object

In [175]:
df3 = my_set_category(df2.dropna().drop('car_name',axis=1))

df3.dtypes

seats                  float64
kms_driven               int64
mileage(kmpl)          float64
engine(cc)             float64
max_power(bhp)         float64
torque(Nm)             float64
price                  float64
registration_year     category
insurance_validity    category
fuel_type             category
ownsership            category
transmission          category
manufacturing_year    category
brand                 category
month                  float64
year                   float64
dtype: object

In [176]:
import numpy as np
a = (int(df3['year'].max())-np.array(df3['year']) ) *12 + df3['month']
df3['registration'] = a
df3['mileage'] = df3['mileage(kmpl)']
df3['engine'] = df3['engine(cc)']
df3['max_power'] = df3['max_power(bhp)']
df3['torque'] = df3['torque(Nm)']
df3.drop(['torque(Nm)','engine(cc)','max_power(bhp)','mileage(kmpl)'],axis=1,inplace=True)
df3['kms_driven2']=df3['kms_driven']**2
df3['registration2'] = df3['registration']**2
df3['torque2'] = df3['torque']**2
df3['max_power2'] = df3['max_power']**2
df3['engine2'] = df3['engine']**2
df3['mileage2'] = df3['mileage'] **2

In [177]:

df4 =df3.drop(['year','month','manufacturing_year','registration_year'],axis=1)
df4.head()


,seats,kms_driven,price,insurance_validity,fuel_type,ownsership,transmission,brand,registration,mileage,engine,max_power,torque,kms_driven2,registration2,torque2,max_power2,engine2,mileage2
1,5.0,30615,8.99,Comprehensive,Petrol,First Owner,Automatic,Nissan,25.0,17.40,999.0,999.0,9863.0,937278225,625.0,97278769.0,998001.0,998001.0,302.7600
2,5.0,24000,23.75,Comprehensive,Diesel,First Owner,Automatic,BMW,69.0,20.68,1995.0,1995.0,188.0,576000000,4761.0,35344.0,3980025.0,3980025.0,427.6624
3,5.0,18378,13.56,Comprehensive,Petrol,First Owner,Manual,Kia,60.0,16.50,1353.0,1353.0,13808.0,337750884,3600.0,190660864.0,1830609.0,1830609.0,272.2500
4,5.0,44900,24.00,Comprehensive,Petrol,First Owner,Automatic,Skoda,56.0,14.67,1798.0,1798.0,17746.0,2016010000,3136.0,314920516.0,3232804.0,3232804.0,215.2089
5,5.0,42000,5.45,Third Party insurance,Petrol,First Owner,Manual,Honda,79.0,18.70,1199.0,1199.0,887.0,1764000000,6241.0,786769.0,1437601.0,1437601.0,349.6900


In [178]:
categories = [x for x in df4.columns if df4[x].dtypes == 'category']


In [179]:
for c in categories:
    tmp = df4[[c, 'price']]
    my_anova(tmp, hue=c, target='price')

pingouin.anova
+----+--------------------+---------+------+---------+---------+----------+------------+
|    | Source             |      SS |   DF |      MS |       F |    p-unc |        np2 |
|----+--------------------+---------+------+---------+---------+----------+------------|
|  0 | insurance_validity | 92.5015 |    2 | 46.2508 | 1.31372 | 0.269407 | 0.00332745 |
|  1 | Within             | 27707.1 |  787 | 35.2059 |     nan |      nan |        nan |
+----+--------------------+---------+------+---------+---------+----------+------------+

statsmodels.anova.anova_lm
+-----------------------+------+----------+-----------+---------+----------+
|                       |   df |   sum_sq |   mean_sq |       F |   PR(>F) |
|-----------------------+------+----------+-----------+---------+----------|
| C(insurance_validity) |    2 |  92.5015 |   46.2508 | 1.31372 | 0.269407 |
| Residual              |  787 |  27707.1 |   35.2059 |     nan |      nan |
+-----------------------+------+------

In [180]:
df_brand = df4[['brand','price']]
df_brand_name = df_brand.groupby('brand').mean().sort_values('price').reset_index()['brand']
map_brand = {}
for i,v in enumerate(df_brand_name):
    map_brand[v] = i//5
df4['brand_value'] = df4['brand'].map(map_brand)
df4.drop('brand',axis=1,inplace=True)

In [181]:
std_df = my_standard_scaler(df4.drop(['insurance_validity','ownsership'],axis=1), 'price')

In [182]:
dummy_df = my_dummies(std_df)

In [183]:
x_train, x_test, y_train, y_test = my_train_test_split(dummy_df, "price", test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((632, 16), (158, 16), (632,), (158,))

In [184]:
# 0행렬이 포함되어있는경우 `LinAlgError: Singular matrix` 발생, 해결방법은 해당 행렬을 찾아서 삭제한다.
# 종속성이 해결이 안될경우 0행렬을 삭제해도 Singular matrix문제 발생 => 해당경우의 해결방법은?

ignore = []
for i in x_train.columns:
    check = False
    for j in x_train[i]:
        check = check or j
        if check:
            break
    else:
        ignore.append(i)
    check = False
    for j in x_test[i]:
        check = check or j
        if check:
            break 
    else:
        ignore.append(i)
ignore

[]

In [185]:
x_train2 = x_train.drop(ignore,axis=1)
x_test2 = x_test.drop(ignore,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test,use_plot=False,resid_test=False)

[회귀식]
price = 0.424 * seats + -0.950 * kms_driven + -0.242 * registration + 4.219 * mileage + 3.653 * engine + 3.653 * max_power + -0.619 * torque + 0.771 * kms_driven2 + -0.927 * registration2 + 0.531 * torque2 + -3.056 * max_power2 + -3.056 * engine2 + -3.627 * mileage2 + 2.040 * brand_value + -0.721 * fuel_type_Petrol + -3.082 * transmission_Manual + 10.826

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.670061 |             2.56164 |             13.7082 |          3.70247 |                         35.2471 |              -7.16193 |
+--------+----------------+---------------------+------------------

LinearRegression(n_jobs=-1)

In [186]:
ignore1 = ignore + ['engine','max_power2','mileage','max_power','torque2','registration','kms_driven2','torque','kms_driven','mileage2','engine2','seats']
x_train2 = x_train.drop(ignore1,axis=1)
x_test2 = x_test.drop(ignore1,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test,use_plot=False,resid_test=False)


[회귀식]
price = -1.252 * registration2 + 2.513 * brand_value + -2.381 * fuel_type_Petrol + -3.347 * transmission_Manual + 12.168

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.644371 |             2.75518 |             14.7756 |          3.84391 |                         37.4006 |              -11.8233 |
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+

[선형회귀분석 결과보고]
+----+------------+---------------------+--------------------+------------+------------------+--------+------------+---------+
|    | 종속변수 

LinearRegression(n_jobs=-1)